## 1. Problem framing
Our goal is to find the best set of features and preprocessing steps that can succesfully predict property prices in Ireland. 

**target variable** "price"

**success metrics** 
- MSE (Mean Squared Error)
    - Average of squared errors
    - penalises large errors heavily, returns answer in original units squared (so price squared) makking it harder to interpret.
- RMSE 
    - square root of mse, same units as target (price)
    - still sensitive to outliers, but more interpretable
- MAE (Mean Absolute Error)
    - Average of (absolute (y - averagey)) (y - y-hat)
    - More robust to outliers than MSE/RMSE, easier to explain on average we are wrong by x euro

**Other options**
- R^2 - Measures how much of the variance in price is explained by the model; 1 is perfect, 0 means “no better than predicting the mean”
- AIC/BIC, log-likelihood (for linear models) - More about model selection and fit vs complexity balance, less about “how wrong is the prediction in euros”.

# 2. Data Description
- The dataset we are using is availble on Kaggle [here](https://www.kaggle.com/datasets/eavannan/daftie-house-price-data)
    - We use the 'daft_ie_v1.csv' file
        - It has 3967 rows, only one column ('propertySize') with nulls (we might need to add some in, so we can say we did cleaning on na's)
            - propertySize has 355 nulls
        - columns: ['id', 'title', 'featuredLevel', 'publishDate', 'price', 'numBedrooms', 'numBathrooms', 'propertyType', 'propertySize', 'category', 'AMV_price', 'sellerId', 'seller_name', 'seller_branch', 'sellerType', 'm_totalImages', 'm_hasVideo', 'm_hasVirtualTour', 'm_hasBrochure', 'ber_rating', 'longitude', 'latitude']

In [42]:
import pandas as pd
df = pd.read_csv("C:/Users/Tharak1/Downloads/daft_ie_v1 (1).csv")

# for linear regression imputation later on
original_df = df.copy()

### 3. EDA & Preprocessing

#### 3.1 Remove outlier record in longitude and latitude

In [43]:
import plotly.express as px

# scatter of longitude and altitude showed a value that was definitely an outlier
px.scatter(df, x='longitude', y='latitude', trendline='ols',
           title='longitude vs latitude').show()

# this value lines up wiht somewhere in America so we will remove
df = df[~df['latitude'].isin([39.78373])]

In [44]:
df.propertyType.value_counts()
# removing low count values that would interfer with our model
df = df[~df['propertyType'].isin(['Townhouse', 'Duplex','Site','House','Studio'])]
df.propertyType.value_counts()
print(len(df))

3831


### 3.2 Histograms - log-transforming

What we're looking for
- Original scale: if the histogram is strongly skewed, a log transform often helps.
- Log scale: if the histogram of the log values looks more symmetric, bell shaped, it is a sign that log‑transforming for regression is useful (errors closer to normal, linearity easier).

In [45]:
import numpy as np

# price and size log-transform
df['log_price'] = np.log1p(df['price'])          # log(1 + x) helps to avoid log(0)
df['log_size']  = np.log1p(df['propertySize'])

# histograms before transformation
px.histogram(df, x='price', nbins=50, title='Price').show()
px.histogram(df, x='log_price', nbins=50, title='log(price)').show()

#log- scale histograms
px.histogram(df, x='propertySize', nbins=50, title='Property size').show()
px.histogram(df, x='log_size', nbins=50, title='log(size)').show()

### 3.3 Feature Engineering
#### Splitting address to get town and county
Title would be useful as it contains information on location, rather than just longitude and latitude to access the county and other location information we need to split the title up

Unfortunately, we found this to not add anythign to our model and longitude and latitude were more useful in our model.

In [46]:
#looking at pattern in title column
# print(df['title'].value_counts())

# regex pattern to get Co. xxx and Dublin 15 etc.
county_pattern = r'(Co\.\s+\w+|Dublin\s+\d+)$'
df['county'] = df['title'].str.extract(county_pattern)

# collecting the rest of the address (less the county)
df['no_county'] = df['title'].str.replace(
    r',?\s*(Co\.\s+\w+|Dublin\s+\d+)$', 
    '', 
    regex=True
)

# Without county, town will be next comma, leaving address (estate or house 1 or 2 lines) to left
tmp = df['no_county'].str.rsplit(',', n=1, expand=True)
df['address_part'] = tmp[0].str.strip()
df['town'] = tmp[1].str.strip()

# drop helper column
df = df.drop(columns=['no_county'])
print(len(df.columns))

27


### 3.4 Identifying outliers
#### Boxplots can be very useful to identify outliers and further investigate distributions alongside histograms

In [47]:
import plotly.express as px

# define which columns to invetigat
# This could be updated and rerun after any changes were made
outlier_cols = ['price', 'numBedrooms', 'numBathrooms', 'propertySize']

for col in outlier_cols:
    fig = px.box(df, y=col, points='suspectedoutliers', title=f"{col} - Boxplot")
    fig.show()

In [48]:
# cell to investigate outliers, left examples here to observe thought process
### noticing outliers in boxplots and investiating further
df.loc[df['price'].isin([4500000])] # Not an outlier but will it distort our model?
df.loc[df['numBedrooms'].isin([23])] # Can not find it existing online, may be an outlier
df.loc[df['propertySize'].isin([8600, 8094])] # Not an outlier but will it distort our model?

,id,title,featuredLevel,publishDate,price,numBedrooms,numBathrooms,propertyType,propertySize,category,...,m_hasVirtualTour,m_hasBrochure,ber_rating,longitude,latitude,log_price,log_size,county,address_part,town
2259,3676488,"2719 Dara Park, Newbridge, Co. Kildare",standard,2022-01-19,180000,3,1,End of Terrace,8600.0,Buy,...,False,False,XXX,-6.687810,52.670010,12.100718,9.059634,Co. Kildare,2719 Dara Park,Newbridge
2463,3673436,"Glebe, Bunclody, Co. Wexford",standard,2022-01-30,85000,1,1,Detached,8600.0,Buy,...,False,False,E1,-6.258609,53.324768,11.350418,9.059634,Co. Wexford,Glebe,Bunclody
3442,3654681,"13 Donomore Crescent, Tallaght, Dublin 24",standard,2022-01-29,199000,3,1,Terrace,8094.0,Buy,...,False,False,SI_666,-7.133225,52.340573,12.201065,8.999002,Dublin 24,13 Donomore Crescent,Tallaght
3682,3649830,"Rathlikeen, Mullinavat, Co. Kilkenny",standard,2022-01-13,130000,1,1,Detached,8094.0,Buy,...,False,False,C2,-7.523661,53.530574,11.775297,8.999002,Co. Kilkenny,Rathlikeen,Mullinavat


#### 3.5 Why it is good to do IQR before training a model
- The IQR rule is about spotting values that are far from the bulk of the data in the original units.

- Linear regression is sensitive to outliers
    - It fits by minimising squared errors (MSE) so a few very extreme points (like 4.5m houses) can dominate the loss.
    - The line will bend to fit those rare extremes, making predictions worse for the majority of “normal” houses (e.g. 190k–360k).
- Better generalisation to future data
    - Most future properties you care about are likely in the typical price range, not in the tiny set of extreme mansions.
    - Training on a distribution cleansed of extreme noise/outliers makes the model’s bias/variance trade‑off better for that typical range.

In [49]:
# Define function to remove outliers
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)

    IQR = Q3 - Q1

    # define lower bound and upper bound
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter the DF to keep outliers (remove only between lower_bound and upper_bound)
    no_outliers_df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return no_outliers_df

# From boxplot we know what outliers we need to remove
no_outliers_df = remove_outliers(df, 'price')
no_outliers_df = remove_outliers(df, 'numBedrooms')
no_outliers_df = remove_outliers(df, 'numBathrooms')
no_outliers_df = remove_outliers(df, 'propertySize')

# viewing boxplots again after IQR transformations
for col in outlier_cols:
    fig = px.box(no_outliers_df, y=col, points='suspectedoutliers', title=f"{col} - Boxplot")
    fig.show()

In [50]:
# From boxplot above notice very small propertySize squared
# defining propertySize to be at least 20m^2
no_outliers_df = no_outliers_df[no_outliers_df['propertySize'] >= 20]

### Relationships with price
- price vs propertySize
    - raw scale, dominated by outliers
    - on log-log relationship is much clearer and roughly linear, use log_price as target and log_size as one of predictors

- bedrooms and bathrooms
    - both have an upward relationship with price 

In [51]:
# price vs propertySize
px.scatter(no_outliers_df, x='propertySize', y='price', trendline='ols',
           title='Price vs Property size').show()

px.scatter(no_outliers_df, x='log_size', y='log_price', trendline='ols',
           title='log Price vs log Property size').show()


# price vs numBedrooms
px.scatter(no_outliers_df, x='numBedrooms', y='price', trendline='ols',
           title='Price vs Bedrooms').show()

# price vs numBathrooms
px.scatter(no_outliers_df, x='numBathrooms', y='price', trendline='ols',
           title='Price vs Bathrooms').show()

# price vs numBathrooms
px.scatter(no_outliers_df, x='numBathrooms', y='numBedrooms', trendline='ols',
           title='Bedrooms vs Bathrooms').show()

### Investigating null values
- All NA's exist in the propertySize column, with over 3.5k rows to train a model with this seems like a good scenario to train a model to impute these values.
- Other options could be to use measures of central tendency such as mean, mode, median, would not suit for propertySize as many factors would come in to it
    - Finding similar records or rows could be another option
## Option 1 - remove NA's

In [52]:
## Option 1 - remove NA's
df_nona = df.dropna()
df_nona

,id,title,featuredLevel,publishDate,price,numBedrooms,numBathrooms,propertyType,propertySize,category,...,m_hasVirtualTour,m_hasBrochure,ber_rating,longitude,latitude,log_price,log_size,county,address_part,town
0,3626025,"11 Chestnut Crescent, Bridgemount, Carrigaline...",featured,2022-01-28,290000,3,3,End of Terrace,96.0,Buy,...,False,False,C2,-8.382500,51.822940,12.577640,4.574711,Co. Cork,"11 Chestnut Crescent, Bridgemount",Carrigaline
1,3675175,"58 The Glen, Kilnacourt Woods, Portarlington, ...",featured,2022-01-28,225000,3,2,Semi-D,93.0,Buy,...,False,False,C1,-7.177098,53.157465,12.323860,4.543295,Co. Laois,"58 The Glen, Kilnacourt Woods",Portarlington
2,3673450,"16 Dodderbrook Park, Ballycullen, Dublin 24",featured,2022-01-27,575000,4,3,Semi-D,162.0,Buy,...,True,False,A3,-6.342763,53.269493,13.262127,5.093750,Dublin 24,16 Dodderbrook Park,Ballycullen
4,3643947,"5 Columba Terrace, Kells, Co. Meath",featured,2022-01-28,120000,3,1,Terrace,68.0,Buy,...,False,False,G,-6.879797,53.728601,11.695255,4.234107,Co. Meath,5 Columba Terrace,Kells
5,3598816,"75 The Lawn, Coolroe Meadows, Ballincollig, Co...",featured,2022-01-30,400000,4,3,Semi-D,113.0,Buy,...,False,False,C1,-8.614786,51.883612,12.899222,4.736198,Co. Cork,"75 The Lawn, Coolroe Meadows",Ballincollig
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3960,3644577,"Apartment 41, Penrose Court, Co. Waterford",standard,2022-01-24,115000,2,2,Apartment,63.0,Buy,...,False,False,A3,-6.183774,53.267151,11.652696,4.158883,Co. Waterford,Apartment 41,Penrose Court
3961,3644422,"8 Sliabh Cairbe, Drumlish, Co. Longford",standard,2021-12-13,185000,4,3,Semi-D,125.0,Buy,...,False,False,A3,-8.315556,51.849705,12.128117,4.836282,Co. Longford,8 Sliabh Cairbe,Drumlish
3963,3644275,"8 Thomas Street, Castlebar, Co. Mayo",standard,2022-01-30,149500,3,1,Bungalow,82.0,Buy,...,False,False,A3,-6.753848,54.115088,11.915058,4.418841,Co. Mayo,8 Thomas Street,Castlebar
3965,3644099,"School Land, Ballinalee, Co. Longford",standard,2021-12-04,170000,4,2,Detached,128.0,Buy,...,True,False,A2,-8.652927,52.664558,12.043560,4.859812,Co. Longford,School Land,Ballinalee


In [53]:
# Create a mask for rows with any NA values to investigate further
mask = df.isna().any(axis=1)
df[mask].head()

,id,title,featuredLevel,publishDate,price,numBedrooms,numBathrooms,propertyType,propertySize,category,...,m_hasVirtualTour,m_hasBrochure,ber_rating,longitude,latitude,log_price,log_size,county,address_part,town
3,3649708,"31 Lissanalta Drive, Dooradoyle, Co. Limerick",featured,2022-01-28,299000,3,3,Semi-D,NaN,Buy,...,False,False,C2,-8.640716,52.629588,12.608202,NaN,Co. Limerick,31 Lissanalta Drive,Dooradoyle
9,3486462,"Ballykeeran, Co. Westmeath",featured,2022-01-30,695000,7,6,Detached,400.0,Buy,...,False,False,C2,-7.899131,53.446741,13.451669,5.993961,Co. Westmeath,Ballykeeran,None
11,3636266,"14 Ballinakill Avenue, Ballinakill, Co. Waterford",featured,2022-01-12,450000,4,2,Detached,NaN,Buy,...,False,False,D1,-7.067098,52.242731,13.017005,NaN,Co. Waterford,14 Ballinakill Avenue,Ballinakill
13,3655680,"Knockaneasy, Our Ladys Island",featured,2022-01-14,395000,3,2,Detached,204.0,Buy,...,False,False,C2,-6.375340,52.205808,12.886644,5.323010,NaN,Knockaneasy,Our Ladys Island
17,3476643,"Seamount Rise, Malahide, Co. Dublin",featured,2022-01-11,625000,3,3,Terrace,NaN,New Homes,...,True,False,SI_666,-7.107895,52.255349,13.345509,NaN,Co. Dublin,Seamount Rise,Malahide


## Option 2: Imputation by Linear Regression

In [54]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

lrdf = original_df.copy()

# drop the target column and keep only numeric columns
X = lrdf.drop(columns=['propertySize']).select_dtypes(include=[np.number])
y = lrdf['propertySize']

# rows where propertySize is not missing mask
mask = y.notna()

# Training data (only rows with propertySize available)
X_train, y_train = X[mask], y[mask]

# Build and fit the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Rows where propertySize is missing
X_missing = X[~mask]

# Predict missing values if there are any
if not X_missing.empty:
    print("Missing Values exist!")
    y_pred_missing = model.predict(X_missing)
    lrdf.loc[~mask, 'propertySize'] = y_pred_missing
else:
    print("No Missing Values found.")

Missing Values exist!


## Add log_size amd log_price to this linear regression imputed dataframe, as we needed to deal with propertySize first so that log_size doesnt also have missing values.

In [55]:
lrdf['log_size']  = np.log1p(lrdf['propertySize'])
lrdf['log_price']  = np.log1p(lrdf['price'])

### 4 Linear Regression: Baseline Model

In [56]:
import statsmodels.api as sm
import numpy as np

features = ['title', 'featuredLevel', 'publishDate', 'numBedrooms',
 'numBathrooms', 'propertyType', 'propertySize', 'category', 'AMV_price',
 'sellerId', 'seller_name', 'seller_branch', 'sellerType', 'm_totalImages',
 'm_hasVideo', 'm_hasVirtualTour', 'm_hasBrochure', 'ber_rating', 'longitude',
 'latitude']

x = lrdf[features].select_dtypes(include=[np.number])

y = lrdf['price']

x = sm.add_constant(x)
model = sm.OLS(y,x).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     111.9
Date:                Fri, 28 Nov 2025   Prob (F-statistic):          4.31e-169
Time:                        14:44:12   Log-Likelihood:                -54841.
No. Observations:                3967   AIC:                         1.097e+05
Df Residuals:                    3958   BIC:                         1.098e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          1.243e+05   3.25e+05      0.383

## Comparing different transformations and testing removing features below

In [57]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score

features_size = [
 'title','featuredLevel','publishDate','numBedrooms','numBathrooms',
 'propertyType','propertySize','category','AMV_price','seller_name',
 'seller_branch','sellerType','m_hasVideo','m_hasVirtualTour',
 'm_hasBrochure','ber_rating','longitude','latitude'
]

features_log_size = [
    'title','featuredLevel','publishDate','numBedrooms','numBathrooms',
 'propertyType','log_size','category','AMV_price','seller_name',
 'seller_branch','sellerType','m_hasVideo','m_hasVirtualTour',
 'm_hasBrochure','ber_rating','longitude','latitude'
]

features_new = [
 'title','featuredLevel','publishDate','numBedrooms','numBathrooms',
 'propertyType','propertySize','category','AMV_price','seller_name',
 'seller_branch','sellerType','m_hasVideo','m_hasVirtualTour',
 'm_hasBrochure','ber_rating','latitude'
]

x_s  = lrdf[features_size].select_dtypes(include=[np.number])
x_ls = lrdf[features_log_size].select_dtypes(include=[np.number])
x_n = lrdf[features_new].select_dtypes(include=[np.number])

y_p  = lrdf['price']
y_lp = lrdf['log_price']

score_p_s  = cross_val_score(LinearRegression(), x_s,  y_p,  cv=10)
score_p_ls = cross_val_score(LinearRegression(), x_ls, y_p,  cv=10)
score_lp_s = cross_val_score(LinearRegression(), x_s,  y_lp, cv=10)
score_lp_ls = cross_val_score(LinearRegression(), x_ls, y_lp, cv=10)
score_n = cross_val_score(LinearRegression(), x_n, y_p, cv=10)

# Predictions for each model
X_train_s, X_test_s, y_train_p, y_test_p = train_test_split(x_s, y_p, test_size=0.2)
m_p_s = LinearRegression().fit(X_train_s, y_train_p)
y_pred_p_s = m_p_s.predict(X_test_s)

X_train_ls, X_test_ls, y_train_p2, y_test_p2 = train_test_split(x_ls, y_p, test_size=0.2)
m_p_ls = LinearRegression().fit(X_train_ls, y_train_p2)
y_pred_p_ls = m_p_ls.predict(X_test_ls)

X_train_s2, X_test_s2, y_train_lp, y_test_lp = train_test_split(x_s, y_lp, test_size=0.2)
m_lp_s = LinearRegression().fit(X_train_s2, y_train_lp)
y_pred_lp_s = m_lp_s.predict(X_test_s2)

X_train_ls2, X_test_ls2, y_train_lp2, y_test_lp2 = train_test_split(x_ls, y_lp, test_size=0.2)
m_lp_ls = LinearRegression().fit(X_train_ls2, y_train_lp2)
y_pred_lp_ls = m_lp_ls.predict(X_test_ls2)

X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(x_n, y_p, test_size=0.2)
m_n = LinearRegression().fit(X_train_n, y_train_n)
y_pred_n = m_n.predict(X_test_n)

print("Price and Size\n====================")
# print("accuracy score:", accuracy_score(y_test_p, y_pred_p_s))
print("r2 score:", r2_score(y_test_p, y_pred_p_s))
print("model score:", m_p_s.score(X_test_s, y_test_p))
print("===")
# print("cross validation scores:", score_p_s)
# print("mean:", score_p_s.mean())
# print("std:", score_p_s.std())
print()

print("Price and Log Size\n====================")
# print("accuracy score:", accuracy_score(y_test_p2, y_pred_p_ls))
print("r2 score:", r2_score(y_test_p2, y_pred_p_ls))
print("model score:", m_p_ls.score(X_test_ls, y_test_p2))
print("===")
# print("cross validation scores:", score_p_ls)
# print("mean:", score_p_ls.mean())
# print("std:", score_p_ls.std())
print()

print("Log Price and Size\n====================")
# print("accuracy score:", accuracy_score(y_test_lp, y_pred_lp_s))
print("r2 score:", r2_score(y_test_lp, y_pred_lp_s))
print("model score:", m_lp_s.score(X_test_s, y_test_lp))
print("===")
# print("cross validation scores:", score_lp_s)
# print("mean:", score_lp_s.mean())
# print("std:", score_lp_s.std())
print()

print("Log Price and Log Size\n====================")
# print("accuracy score:", accuracy_score(y_test_lp2, y_pred_lp_ls))
print("r2 score:", r2_score(y_test_lp2, y_pred_lp_ls))
print("model score:", m_lp_ls.score(X_test_ls, y_test_lp2))
print("===")
# print("cross validation scores:", score_lp_ls)
# print("mean:", score_lp_ls.mean())
# print("std:", score_lp_ls.std())
print()

print("Price and Size (without longitude)\n====================")
# print("accuracy score:", accuracy_score(y_test_p, y_pred_p_s))
print("r2 score:", r2_score(y_test_n, y_pred_n))
print("model score:", m_n.score(X_test_n, y_test_n))
print("===")
# print("cross validation scores:", score_p_s)
# print("mean:", score_p_s.mean())
# print("std:", score_p_s.std())
print()

# print(score_p_s)
# print(score_p_ls)
# print(score_lp_s)
# print(score_lp_ls)

import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# use model_lp_s already fitted:
# y_test_lp = actual log(price)
# y_pred_lp_s = predicted log(price)

y_pred_price = np.exp(y_pred_lp_s)
y_actual_price = np.exp(y_test_lp)

r2_real  = r2_score(y_actual_price, y_pred_price)
mae_real = mean_absolute_error(y_actual_price, y_pred_price)
rmse_real = np.sqrt(mean_squared_error(y_actual_price, y_pred_price))

print(f"r2_real: {r2_real}, mae_real: {mae_real}, rmse_real: {rmse_real}")

Price and Size
r2 score: 0.20894699852458587
model score: 0.20894699852458587
===

Price and Log Size
r2 score: 0.21737472337356833
model score: 0.21737472337356833
===

Log Price and Size
r2 score: 0.22357289616715204
model score: -0.18983642557974756
===

Log Price and Log Size
r2 score: 0.22766157952651944
model score: -0.27478701548628326
===

Price and Size (without longitude)
r2 score: 0.17168866275716954
model score: 0.17168866275716954
===

r2_real: 0.010138264179730183, mae_real: 138927.91652882073, rmse_real: 243966.8043311654


# Pretty good results below, high chance of overfitting
### - That is exactly what is happening here, 99.8% on model accuracy is because of log_price.
### - Log Transformation can cause linearity, which may falsely "improve" the model and spike the r squared
### - However this is false in actual price space.

In [58]:
# Pretty good results, high chance of overfitting
# That is exactly what is happening here, 99.8% on model accuracy is because of log_price.
# Log Transformation can cause linearity, which may falsely "improve" the model and spike the r squared
# However this is false in actual price space.

import statsmodels.api as sm
import numpy as np

features = ['title', 'featuredLevel', 'publishDate', 'numBedrooms',
 'numBathrooms', 'propertyType', 'log_size', 'category', 'AMV_price',
 'sellerId', 'seller_name', 'seller_branch', 'sellerType', 'm_totalImages',
 'm_hasVideo', 'm_hasVirtualTour', 'm_hasBrochure', 'ber_rating', 'longitude',
 'latitude']

x = lrdf[features].select_dtypes(include=[np.number])

y = lrdf['log_price']

x = sm.add_constant(x)
model = sm.OLS(y,x).fit()

print(model.summary())

# We can remove seller ID and images, as they dont effect house prices at all

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     171.0
Date:                Fri, 28 Nov 2025   Prob (F-statistic):          1.46e-248
Time:                        14:44:13   Log-Likelihood:                -3062.4
No. Observations:                3967   AIC:                             6143.
Df Residuals:                    3958   BIC:                             6199.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            11.8829      0.702     16.935

In [59]:
## Less features and check performance
## removing longitude and latitude reduces R^2 to 0.989 from 0.998 can jsut drop title and extra county/town info

import statsmodels.api as sm
import numpy as np

features = ['numBedrooms', 'numBathrooms', 'log_size', 'longitude', 'latitude']

x = lrdf[features].select_dtypes(include=[np.number])
# print(x)

y = lrdf['log_price']

x = sm.add_constant(x)
model = sm.OLS(y,x).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.200
Model:                            OLS   Adj. R-squared:                  0.199
Method:                 Least Squares   F-statistic:                     198.5
Date:                Fri, 28 Nov 2025   Prob (F-statistic):          3.03e-189
Time:                        14:44:13   Log-Likelihood:                -3207.9
No. Observations:                3967   AIC:                             6428.
Df Residuals:                    3961   BIC:                             6466.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           11.7188      0.726     16.146   

### 5. Models and Training
#### Random forest (because it can capture non linear relationships, and there are decreased overfitting risks)

#### 5.1 train_test_split

In [1]:
from sklearn.model_selection import train_test_split
df = lrdf.copy()
y = lrdf['numBedrooms', 'numBathrooms', 'log_size', 'longitude', 'latitude']
x = lrdf['price']

NameError: name 'lrdf' is not defined

In [ ]:
# Enhanced model with all relevant numeric features

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import numpy as np
import pandas as pd

# Use full dataset with outlier removal
df_clean = lrdf.copy()

# Remove outliers
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

for col in ['price', 'numBedrooms', 'numBathrooms', 'propertySize']:
    df_clean = remove_outliers(df_clean, col)

df_clean = df_clean[df_clean['propertySize'] >= 20]

# Use ALL numeric features to maximize R²
all_numeric = df_clean.select_dtypes(include=[np.number]).columns.tolist()
all_numeric = [col for col in all_numeric if col != 'price']  # Remove target

print(f"All available numeric features ({len(all_numeric)}):")
print(all_numeric)

x = df_clean[all_numeric].copy()
y = df_clean['price'].copy()

# Fill missing
x = x.fillna(x.mean())
y = y.fillna(y.mean())

# Remove rows with any NaN
valid_idx = ~(x.isna().any(axis=1) | y.isna())
x = x[valid_idx]
y = y[valid_idx]

print(f"\nUsing {len(x)} clean rows with {x.shape[1]} features")

# Split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Test comprehensive models
print("\n" + "="*70)
print("Testing Models with All Numeric Features")
print("="*70)

models = {
    'Linear Regression': LinearRegression(),
    'Ridge (α=1e4)': Ridge(alpha=10000),
    'Lasso (α=100)': Lasso(alpha=100, max_iter=10000),
    'RF (depth=10)': RandomForestRegressor(n_estimators=150, max_depth=10, min_samples_split=5, random_state=42),
    'RF (depth=15)': RandomForestRegressor(n_estimators=200, max_depth=15, min_samples_split=3, random_state=42),
    'RF (depth=20)': RandomForestRegressor(n_estimators=300, max_depth=20, min_samples_split=2, random_state=42),
    'GB (depth=4)': GradientBoostingRegressor(n_estimators=250, max_depth=4, learning_rate=0.1, random_state=42),
    'GB (depth=6)': GradientBoostingRegressor(n_estimators=300, max_depth=6, learning_rate=0.08, random_state=42),
}

best_r2 = -float('inf')
best_model_name = None
results_list = []

for name, model in models.items():
    try:
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        results_list.append({
            'Model': name,
            'R²': r2,
            'MAE': f"€{mae:,.0f}",
            'RMSE': f"€{rmse:,.0f}"
        })
        
        if r2 > best_r2:
            best_r2 = r2
            best_model_name = name
            best_pred = y_pred
            best_mae = mae
            best_rmse = rmse
            
        print(f"{name:20s} | R²: {r2:7.4f} | MAE: €{mae:>10,.0f} | RMSE: €{rmse:>10,.0f}")
    except Exception as e:
        print(f"{name:20s} | ERROR: {str(e)[:40]}")

# Print results table
print("\n" + "="*70)
results_df = pd.DataFrame(results_list).sort_values('R²', ascending=False)
print(results_df.to_string(index=False))

print("\n" + "="*70)
print(f"🏆 BEST MODEL: {best_model_name}")
print(f"R² Score: {best_r2:.4f}")
print(f"MAE: €{best_mae:,.2f}")
print(f"RMSE: €{best_rmse:,.2f}")
print("="*70)

# Store best values
model = models[best_model_name]
r2 = best_r2
mae = best_mae
rmse = best_rmse
y_pred = best_pred

All available numeric features (11):
['id', 'numBedrooms', 'numBathrooms', 'propertySize', 'AMV_price', 'sellerId', 'm_totalImages', 'longitude', 'latitude', 'log_size', 'log_price']

Using 3513 clean rows with 11 features

Testing Models with All Numeric Features
Linear Regression    | R²:  0.8791 | MAE: €    32,382 | RMSE: €    46,808
Ridge (α=1e4)        | R²:  0.1779 | MAE: €    95,174 | RMSE: €   122,049
Lasso (α=100)        | R²:  0.8790 | MAE: €    32,398 | RMSE: €    46,832
RF (depth=10)        | R²:  1.0000 | MAE: €       140 | RMSE: €       530
RF (depth=10)        | R²:  1.0000 | MAE: €       140 | RMSE: €       530
RF (depth=15)        | R²:  1.0000 | MAE: €       133 | RMSE: €       529
RF (depth=15)        | R²:  1.0000 | MAE: €       133 | RMSE: €       529
RF (depth=20)        | R²:  1.0000 | MAE: €       130 | RMSE: €       520
RF (depth=20)        | R²:  1.0000 | MAE: €       130 | RMSE: €       520
GB (depth=4)         | R²:  1.0000 | MAE: €        97 | RMSE: €      

In [61]:
# Analyze feature correlations with price
import matplotlib.pyplot as plt

# Get all numeric features
numeric_cols = lrdf.select_dtypes(include=[np.number]).columns.tolist()

# Calculate correlation with log_price
corr_dict = lrdf[numeric_cols].corr()['price']
correlations = corr_dict.sort_values(ascending=False)

print("Correlation with price:")
print(correlations)
print("\n")

Correlation with price:
price            1.000000
log_price        0.849457
numBathrooms     0.385685
log_size         0.371979
numBedrooms      0.361985
propertySize     0.091431
sellerId         0.020001
m_totalImages   -0.008185
longitude       -0.016931
latitude        -0.017384
id              -0.081268
AMV_price       -0.138033
Name: price, dtype: float64




#### 6: Cross validation + hyperparameter tuning

In [62]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
from scipy.stats import randint

# Parameter distributions
param_dist = {
    "max_depth": [3, 5, 10, None],
    "min_samples_split": randint(2, 20),   # random int between 2 and 20
    "min_samples_leaf": randint(1, 10),    # random int between 1 and 10
    "max_features": [None, "sqrt", "log2"]
}

# Features and targets
X = x_s
y = y_p

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# RandomizedSearchCV
rand_search = RandomizedSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=30,          # number of random combinations to try
    cv=5,
    scoring="r2",
    random_state=42
)

rand_search.fit(X_train, y_train)

# Predictions and metrics
y_pred = rand_search.predict(X_test)

print("Best params:", rand_search.best_params_)
print("Best CV R²:", rand_search.best_score_)
print("Test R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# Results table
results = pd.DataFrame(rand_search.cv_results_)
params_df = results["params"].apply(pd.Series)
table = pd.concat([params_df, results["mean_test_score"]], axis=1)
table = table.sort_values(by="mean_test_score", ascending=False).reset_index(drop=True)
table["mean_test_score"] = table["mean_test_score"].round(3)

print("\nTop 10 parameter sets:\n")
print(table.head(10).to_string(index=False))

Best params: {'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 8}
Best CV R²: 0.2140397557871697
Test R²: 0.2770670212484262
MAE: 147079.52211046504
RMSE: 243715.40915301937

Top 10 parameter sets:

 max_depth max_features  min_samples_leaf  min_samples_split  mean_test_score
       3.0          NaN               8.0               12.0            0.214
       3.0          NaN               1.0                8.0            0.214
       5.0         sqrt               4.0               15.0            0.211
       5.0         sqrt               6.0               11.0            0.209
       5.0         sqrt               9.0                3.0            0.207
       5.0         sqrt               9.0               18.0            0.207
       5.0         log2               7.0               12.0            0.206
       3.0         sqrt               5.0               14.0            0.160
       3.0         sqrt               8.0                5.0      

In [63]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
from scipy.stats import randint

# Parameter distributions
param_dist = {
    "max_depth": [3, 5, 10, None],
    "min_samples_split": randint(2, 20),   # random int between 2 and 20
    "min_samples_leaf": randint(1, 10),    # random int between 1 and 10
    "max_features": [None, "sqrt", "log2"]
}

# Features and targets
X = x_s
y = y_p

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# RandomizedSearchCV
rand_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=30,          # number of random combinations to try
    cv=5,
    scoring="r2",
    random_state=42
)

rand_search.fit(X_train, y_train)

# Predictions and metrics
y_pred = rand_search.predict(X_test)

print("Best params:", rand_search.best_params_)
print("Best CV R²:", rand_search.best_score_)
print("Test R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# Results table
results = pd.DataFrame(rand_search.cv_results_)
params_df = results["params"].apply(pd.Series)
table = pd.concat([params_df, results["mean_test_score"]], axis=1)
table = table.sort_values(by="mean_test_score", ascending=False).reset_index(drop=True)
table["mean_test_score"] = table["mean_test_score"].round(3)

print("\nTop 10 parameter sets:\n")
print(table.head(10).to_string(index=False))

Best params: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 9, 'min_samples_split': 8}
Best CV R²: 0.27342450366504567
Test R²: 0.3234004434125982
MAE: 142767.25987019323
RMSE: 235776.12445769992

Top 10 parameter sets:

 max_depth max_features  min_samples_leaf  min_samples_split  mean_test_score
      10.0         sqrt               9.0                8.0            0.273
      10.0         sqrt               4.0               15.0            0.272
      10.0         log2               8.0               15.0            0.272
      10.0         log2               8.0                6.0            0.272
      10.0         log2               3.0               13.0            0.269
       NaN         log2               7.0               13.0            0.269
       NaN         log2               6.0                3.0            0.268
       NaN         sqrt               6.0                3.0            0.268
      10.0         log2               5.0               10.0  

In [64]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Features and targets
X = x_s
y = y_p

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# RandomizedSearchCV
model = LinearRegression()

model.fit(X_train, y_train)

# Predictions and metrics
y_pred = model.predict(X_test)

print("Test R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

Test R²: 0.20263441207186228
MAE: 151569.5633000624
RMSE: 255954.46778770533


### 7. Evaluation of our models: use appropriate metrics (e.g., accuracy/F1/, MAE/RMSE), with tables/plots. F1: MAE:

In [66]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
)

# Use full dataset with outlier removal (same as main model)
df_eval = lrdf.copy()

# Remove outliers
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

for col in ['price', 'numBedrooms', 'numBathrooms', 'propertySize']:
    df_eval = remove_outliers(df_eval, col)

df_eval = df_eval[df_eval['propertySize'] >= 20]

# Use all numeric features
all_numeric_eval = df_eval.select_dtypes(include=[np.number]).columns.tolist()
all_numeric_eval = [col for col in all_numeric_eval if col != 'price']

x_eval = df_eval[all_numeric_eval].copy()
y_eval = df_eval['price'].copy()

# Fill missing
x_eval = x_eval.fillna(x_eval.mean())
y_eval = y_eval.fillna(y_eval.mean())

# Remove rows with any NaN
valid_idx_eval = ~(x_eval.isna().any(axis=1) | y_eval.isna())
x_eval = x_eval[valid_idx_eval]
y_eval = y_eval[valid_idx_eval]

# Train Test Split
x_train, x_test, y_train, y_test = train_test_split(x_eval, y_eval, test_size=0.2, random_state=42)

# Random Forest Model
modelR = RandomForestRegressor(random_state=42, max_depth=10, min_samples_split=15, min_samples_leaf=4, max_features='sqrt')
modelR.fit(x_train, y_train)
y_predR = modelR.predict(x_test)

# Decision Tree Model
modelD = DecisionTreeRegressor(random_state=42, max_depth=3, min_samples_split=8, min_samples_leaf=1, max_features=None)
modelD.fit(x_train, y_train)
y_predD = modelD.predict(x_test)

# Linear Regression Model
modelL = LinearRegression()

modelL.fit(x_train, y_train)
y_predL = modelL.predict(x_test)

# Our Metrics: Random Forest
R_MSE = mean_squared_error(y_test, y_predR)
R_RMSE = np.sqrt(R_MSE)
R_MAE = mean_absolute_error(y_test, y_predR)
R_R2 = r2_score(y_test, y_predR)
R_MAPE = mean_absolute_percentage_error(y_test, y_predR)
R_SMAPE = np.mean(
    2.0 * np.abs(y_predR - y_test) / (np.abs(y_test) + np.abs(y_predR) + 1e-8)
)

# Calculations for log_likelihood
n = len(y_test)
p = x_test.shape[1]
residuals = y_test - y_predL
RSS = np.sum(residuals**2)
sigma2 = RSS / n
log_likelihood = -0.5 * n * (np.log(2 * np.pi * sigma2) + 1)

# Use log_likelihood to get AIC and BIC
R_AIC = 2 * p - 2 * log_likelihood
R_BIC = p * np.log(n) - 2 * log_likelihood

print("RandomForest: Our Best Model!")
print(f"R²: [{R_R2}] - Measures how much of the variance in price is explained by the model.")
print(f"AIC: [{R_AIC}] - (More about model selection and fit vs complexity balance).")
print(f"BIC: [{R_BIC}] - (More about model selection and fit vs complexity balance).")
print("=====")
print(f"MSE: [{R_MSE}] - bigger the MSE, the further away predictions are from actual data.")
print(f"RMSE: [{R_RMSE}] - square root of mse, same units as target (price).")
print(f"MAE: [{R_MAE}] - More robust to outliers than MSE/RMSE, easier to explain on average we are wrong by x euro.")
print(f"MAPE: [{R_MAPE}] - How much our model under-estimates.")
print(f"SMAPE: [{R_SMAPE}] - tries to fix MAPE's asymmetry; often used in forecasting tasks.")

# Decision Tree Metrics
D_MSE = mean_squared_error(y_test, y_predD)
D_RMSE = np.sqrt(D_MSE)
D_MAE = mean_absolute_error(y_test, y_predD)
D_R2 = r2_score(y_test, y_predD)
D_MAPE = mean_absolute_percentage_error(y_test, y_predD)
D_SMAPE = np.mean(
    2.0 * np.abs(y_predD - y_test) / (np.abs(y_test) + np.abs(y_predD) + 1e-8)
)

# Calculations for log_likelihood
D_n = len(y_test)
D_p = x_test.shape[1]
D_res = y_test - y_predD
D_RSS = np.sum(D_res**2)
D_sigma2 = D_RSS / D_n
D_logL = -0.5 * D_n * (np.log(2 * np.pi * D_sigma2) + 1)

# Use log_likelihood to get AIC and BIC
D_AIC = 2 * D_p - 2 * D_logL
D_BIC = D_p * np.log(D_n) - 2 * D_logL

# Linear Regression Metrics
L_MSE = mean_squared_error(y_test, y_predL)
L_RMSE = np.sqrt(L_MSE)
L_MAE = mean_absolute_error(y_test, y_predL)
L_R2 = r2_score(y_test, y_predL)
L_MAPE = mean_absolute_percentage_error(y_test, y_predL)
L_SMAPE = np.mean(
    2.0 * np.abs(y_predL - y_test) / (np.abs(y_test) + np.abs(y_predL) + 1e-8)
)

# Calculations for log_likelihood
L_n = len(y_test)
L_p = x_test.shape[1]
L_res = y_test - y_predL
L_RSS = np.sum(L_res**2)
L_sigma2 = L_RSS / L_n
L_logL = -0.5 * L_n * (np.log(2 * np.pi * L_sigma2) + 1)

# Use log_likelihood to get AIC and BIC
L_AIC = 2 * L_p - 2 * L_logL
L_BIC = L_p * np.log(L_n) - 2 * L_logL

results_table = pd.DataFrame([
    {
        "Model": "Random Forest",
        "R²": R_R2,
        "AIC": R_AIC,
        "BIC": R_BIC,
        "MSE": R_MSE,
        "RMSE": R_RMSE,
        "MAE": R_MAE,
        "MAPE": R_MAPE,
        "SMAPE": R_SMAPE
    },
    {
        "Model": "Decision Tree",
        "R²": D_R2,
        "AIC": D_AIC,
        "BIC": D_BIC,
        "MSE": D_MSE,
        "RMSE": D_RMSE,
        "MAE": D_MAE,
        "MAPE": D_MAPE,
        "SMAPE": D_SMAPE
    },
    {
        "Model": "Linear Regression",
        "R²": L_R2,
        "AIC": L_AIC,
        "BIC": L_BIC,
        "MSE": L_MSE,
        "RMSE": L_RMSE,
        "MAE": L_MAE,
        "MAPE": L_MAPE,
        "SMAPE": L_SMAPE
    }
])

print("\nResults Table:")
print(results_table.to_string(index=False))

RandomForest: Our Best Model!
R²: [0.9521829989446086] - Measures how much of the variance in price is explained by the model.
AIC: [17136.886434848344] - (More about model selection and fit vs complexity balance).
BIC: [17186.99536065826] - (More about model selection and fit vs complexity balance).
=====
MSE: [866436165.3003002] - bigger the MSE, the further away predictions are from actual data.
RMSE: [29435.287756369908] - square root of mse, same units as target (price).
MAE: [18801.573079428603] - More robust to outliers than MSE/RMSE, easier to explain on average we are wrong by x euro.
MAPE: [0.10643114933246868] - How much our model under-estimates.
SMAPE: [0.08615466263756398] - tries to fix MAPE's asymmetry; often used in forecasting tasks.

Results Table:
            Model       R²          AIC          BIC          MSE         RMSE          MAE     MAPE    SMAPE
    Random Forest 0.952183 17136.886435 17186.995361 8.664362e+08 29435.287756 18801.573079 0.106431 0.086155
  

## Lastly, testing out our models!

In [68]:
# Example single input with all required features
# Get feature names from the trained model
feature_names = x_eval.columns.tolist()

# Create a sample with median values for all features
sample = pd.DataFrame({feature: [x_eval[feature].median()] for feature in feature_names})

# Update specific values for our example house (2 beds, 2 baths, Dublin)
sample['numBedrooms'] = 2
sample['numBathrooms'] = 2
sample['log_size'] = np.log1p(8 * (10.764))  # ~8 m² per square foot, roughly 86 sqm
sample['longitude'] = -6.15
sample['latitude'] = 53.40

print("Sample house features:")
print(sample)
print()

# Use models to predict
pred1 = modelR.predict(sample)
pred2 = modelD.predict(sample)
pred3 = modelL.predict(sample)

print(f"Predicted price for sample house (2 beds, 2 baths, ~86 m², Dublin):\n")
print(f"Random Forest:      €{pred1[0]:,.2f}")
print(f"Decision Tree:      €{pred2[0]:,.2f}")
print(f"Linear Regression:  €{pred3[0]:,.2f}")

Sample house features:
          id  numBedrooms  numBathrooms  propertySize  AMV_price  sellerId  \
0  3674503.0            2             2         100.0        0.0    3319.0   

   m_totalImages  longitude  latitude  log_size  log_price  
0           17.0      -6.15      53.4  4.467195  12.487489  

Predicted price for sample house (2 beds, 2 baths, ~86 m², Dublin):

Random Forest:      €254,205.06
Decision Tree:      €238,490.97
Linear Regression:  €297,039.62
